# MultiQueryRetriever

하나의 사용자 질의에 대해 다양한 각도의 쿼리를 생성하고 그 결과를 통합함으로써 검색 성능을 높여주는 구성 요소


거리 기반 벡터 데이터베이스 검색은 고차원 공간에서의 쿼리 임베딩(표현)과 '거리'를 기준으로 유사한 임베딩을 가진 문서를 찾는 방식입니다. 하지만 쿼리의 **세부적인 차이나 임베딩이 데이터의 의미를 제대로 포착하지 못할 경우, 검색 결과가 달라질 수** 있습니다. 또한, 이를 수동으로 조정하는 프롬프트 엔지니어링이나 튜닝 작업은 번거로울 수 있습니다.

이런 문제를 해결하기 위해, `MultiQueryRetriever` 는 주어진 사용자 입력 쿼리에 대해 다양한 관점에서 여러 쿼리를 자동으로 생성하는 LLM(Language Learning Model)을 활용해 프롬프트 튜닝 과정을 자동화합니다.

이 방식은 각각의 쿼리에 대해 관련 문서 집합을 검색하고, 모든 쿼리를 아우르는 고유한 문서들의 합집합을 추출해, 잠재적으로 관련된 더 큰 문서 집합을 얻을 수 있게 해줍니다. 

여러 관점에서 동일한 질문을 생성함으로써, `MultiQueryRetriever` 는 거리 기반 검색의 제한을 일정 부분 극복하고, 더욱 풍부한 검색 결과를 제공할 수 있습니다.

In [ ]:
# 샘플 벡터DB 구축
from dotenv import load_dotenv                                        # API 키를 환경변수로 관리하기 위한 설정 파일
from langchain_community.document_loaders import WebBaseLoader        # 웹 페이지에서 HTML 텍스트를 로드
from langchain.vectorstores import FAISS                              # FAISS(Facebook AI Similarity Search): 빠른 유사도 검색을 위한 벡터 저장소
from langchain_openai import OpenAIEmbeddings                         # OpenAI의 텍스트 임베딩 모델 (텍스트 → 벡터로 변환)
from langchain_text_splitters import RecursiveCharacterTextSplitter   # 문서를 일정 길이로 잘라서 분할할

# 블로그 포스트 로드
loader = WebBaseLoader(
    "https://medium.com/naver-cloud-platform/cloud-web-service-monitoring-system-wms-v2-0-a-to-z-1e29069598b2", encoding="utf-8"
)

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = loader.load_and_split(text_splitter)

# 임베딩 정의
openai_embedding = OpenAIEmbeddings()

# 벡터DB 생성
db = FAISS.from_documents(docs, openai_embedding)

# retriever 생성
retriever = db.as_retriever()

# 문서 검색
query = "WMS의 주요 기능에 대해 알려주세요."
relevant_docs = retriever.invoke(query)

# 검색된 문서의 개수 출력
len(relevant_docs)

4

검색된 결과 중 1개 문서의 내용을 출력합니다.


In [8]:
# 1번 문서를 출력합니다.
print(relevant_docs[1].page_content)

API와 SDK 정보를 확인하실 수 있습니다.​글을 마무리하며…네이버 클라우드 플랫폼의 WMS 상품 및 API을 활용한 쉽고 빠른 웹 서비스 모니터링 방법을 공유드렸습니다. 네이버 클라우드 플랫폼의 WMS 상품을 활용하여 저비용/고성능의 서비스 품질 향상을 이루고 빠른 장애 대응 방법을 적용한다면 사용자의 서비스 만족도를 극대화하실 수 있습니다.앞으로도 지속적인 WMS 기능과 성능 향상을 통해 보다 최적화 및 고도화된 웹 서비스 모니터링을 경험할 수 있도록 서비스를 발전시켜 나갈 예정입니다.많은 관심 부탁드리며, 끝까지 읽어 주셔서 감사합니다.본 기술 포스팅은 네이버클라우드 S/W Development Engineer 장창환 님의 작성 글입니다.Web MonitoringWmsSynthetic MonitoringWeb DevelopmentReal User Monitoring----FollowPublished in NAVER Cloud440 followers·Last





`MultiQueryRetriever` 에 사용할 LLM을 지정하고 질의 생성에 사용하면, retriever가 나머지 작업을 처리합니다.


In [9]:
from langchain.retrievers.multi_query import MultiQueryRetriever  # 하나의 질문에 대해 다양한 방식으로 재질문하여 더 풍부한 검색 결과를 제공하는 Retriever
from langchain_openai import ChatOpenAI                           # OpenAI의 GPT 기반 대화형 언어 모델 (예: gpt-3.5-turbo, gpt-4 등)을 사용하기 위한 래퍼

# ChatOpenAI 언어 모델을 초기화합니다. temperature는 0으로 설정합니다. 창의성X
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

multiquery_retriever = MultiQueryRetriever.from_llm(  # MultiQueryRetriever를 언어 모델을 사용하여 초기화합니다.
    # 벡터 데이터베이스의 retriever와 언어 모델을 전달합니다.
    retriever=db.as_retriever(),
    llm=llm,
)

아래는 다중 쿼리를 생성하는 중간 과정을 디버깅하기 위하여 실행하는 코드입니다.

먼저 `"langchain.retrievers.multi_query"` 로거를 가져옵니다. 

이는 `logging.getLogger()` 함수를 사용하여 수행됩니다. 그 다음, 이 로거의 로그 레벨을 `INFO`로 설정하여, `INFO` 레벨 이상의 로그 메시지만 출력되도록 할 수 있습니다. 


In [10]:
# 쿼리에 대한 로깅 설정
import logging                  # 로깅 기능을 사용하기 위한 logging 모듈 임포트트

logging.basicConfig()           # 기본 로깅 설정을 초기화 (포맷/출력 대상 등 기본 설정)

# 'langchain.retrievers.multi_query' 모듈의 로깅 수준을 INFO로 설정
# → 다양한 서브 쿼리 생성 과정을 콘솔에서 확인 가능함
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

이 코드는 `retriever_from_llm` 객체의 `invoke` 메서드를 사용하여 주어진 `question`과 관련된 문서를 검색합니다. 

검색된 문서들은 `unique_docs`라는 변수에 저장되며, 이 변수의 길이를 확인함으로써 검색된 관련 문서의 총 개수를 알 수 있습니다. 이 과정을 통해 사용자의 질문에 대한 관련 정보를 효과적으로 찾아내고 그 양을 파악할 수 있습니다.


In [11]:
# 질문을 정의합니다.
question = "WMS의 주요 기능에 대해 알려주세요."
# 문서 검색
relevant_docs = multiquery_retriever.invoke(question)

# 검색된 고유한 문서의 개수를 반환합니다.
print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

# 검색된 문서의 내용을 출력합니다.
print(relevant_docs[0].page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['WMS의 핵심 기능은 무엇인가요?  ', 'WMS 시스템에서 제공하는 주요 기능들은 어떤 것들이 있나요?  ', 'WMS의 기능적 특징에 대해 설명해 주세요.']


검색된 문서 개수: 5
플랫폼만의 컴포넌트 구조를 특화하여 안정적인 웹 서비스 모니터링을 지원하도록 구성되었습니다.<그림 31> WMS Service Architecture — Simple Version(1) WMS Agent Layer사용자가 모니터링을 위해 등록한 URL을 주기적으로 실행하고, 데이터를 수집하는 역할을 담당하고 있습니다. 해당 Agent Instance에서 Headless Browser Module을 실행하여 Web Service Object에 대한 정보를 수집합니다.이렇게 수집된 데이터는 Data Uploader를 통해 WMS Collector로 전송됩니다.(2) WMS Collector Layer각 WMS Agent에 전송된 Object Data를 각 목적에 맞게 데이터를 분류하고, 분석을 진행 후 Data Storage로 저장합니다. 그리고 사용자의 설정에 의해 발생된 이벤트를 감지하고 알람 메지지를 전송하는 역할을 수행합니다.(3) WMS Aggregator LayerWMS


## LCEL Chain 활용하는 방법

LangChain Expression Language (LCEL)은 LangChain에서 체인(Chain)을 더 간결하고 선언적으로 구성할 수 있도록 만든 표현식 기반 체인 구성 방식

- 사용자 정의 프롬프트 정의하고, 정의한 프롬프트와 함께 Chain 을 생성합니다.
- Chain 은 사용자의 질문을 입력 받으면 (아래의 예제에서는) 5개의 질문을 생성한 뒤 `"\n"` 구분자로 구분하여 생성된 5개 질문을 반환합니다.


In [ ]:
from langchain_core.runnables import RunnablePassthrough       # 입력을 그대로 다음 단계로 넘기는 역할을 하는 Runnable
from langchain_core.prompts import PromptTemplate              # 프롬프트 템플릿을 정의하는 클래스 (사용자 입력을 LLM에 맞는 형식으로 가공)
from langchain_core.output_parsers import StrOutputParser      # LLM의 출력 결과를 문자열로 파싱해주는 기본 파서 (ex. JSON, 리스트가 아닌 그냥 텍스트를 다룰 때)

# 프롬프트 템플릿을 정의합니다.(5개의 질문을 생성하도록 프롬프트를 작성하였습니다)
prompt = PromptTemplate.from_template(
    """당신은 AI 언어 모델 어시스턴트입니다.
당신의 작업은 주어진 사용자 질문에 대해 관련 문서를 벡터 데이터베이스에서 검색할 수 있도록 다섯 가지 다른 버전의 질문을 생성하는 것입니다.
질문에 대한 다양한 관점을 생성함으로써, 거리 기반 유사도 검색의 한계를 극복하는 데 도움을 주는 것이 목표입니다.
당신의 응답은 다음과 같은 형식으로, 각 질문을 줄바꿈(\n)으로 구분된 값 목록으로 출력해야 합니다.

#ORIGINAL QUESTION: 
{question}

"""
)

# 언어 모델 인스턴스를 생성합니다.
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

# LLMChain을 생성합니다.
# 사용자로부터 받은 질문을 LLM에 맞는 프롬프트로 바꾸고
# OpenAI LLM에 전달해서 답변을 받고
# 그 결과를 문자열로 출력하는 전체 흐름을 구성

custom_multiquery_chain = (
    {"question": RunnablePassthrough()} | prompt | llm | StrOutputParser()
)

# 질문을 정의합니다.
question = "Web service Monitoring System 2.0의 주요 기능은?"

# 체인을 실행하여 생성된 다중 쿼리를 확인합니다.
multi_queries = custom_multiquery_chain.invoke(question)
# 결과를 확인합니다.(5개 질문 생성)
multi_queries

'Web service Monitoring System 2.0에서 제공하는 핵심 기능은 무엇인가요?  \nWeb service Monitoring System 2.0의 기능적 특징은 어떤 것들이 있습니까?  \nWeb service Monitoring System 2.0의 주요 기능 목록은 무엇인지 알고 싶습니다.  \nWeb service Monitoring System 2.0에서 지원하는 주요 기능은 무엇인가요?  \nWeb service Monitoring System 2.0의 기능은 어떤 점에서 유용한가요?  '

이전에 생성한 Chain을 `MultiQueryRetriever` 에 전달하여 retrieve 할 수 있습니다.


In [ ]:
multiquery_retriever = MultiQueryRetriever.from_llm(     # 위에서 생성한 custom_multiquery_chain을 multiquery_retriever에 적용
    llm=custom_multiquery_chain, retriever=db.as_retriever()
)

`MultiQueryRetriever`를 사용하여 문서를 검색하고 결과를 확인합니다.


In [ ]:
# 결과
# 사용자 입력 쿼리 > 프롬프트 템플릿 적용하여 LLM을 통해 다양한 버전의 질문 생성 > FAISS 벡터 DB에서 검색수행
relevant_docs = multiquery_retriever.invoke(question)

# 검색된 고유한 문서의 개수를 반환합니다.
print(
    f"===============\n검색된 문서 개수: {len(relevant_docs)}",
    end="\n===============\n",
)

# 검색된 문서의 내용을 출력합니다.
print("\n\n".join([doc.page_content for doc in relevant_docs]))
#print(relevant_docs[0].page_content)

INFO:langchain.retrievers.multi_query:Generated queries: ['Web service Monitoring System 2.0의 핵심 기능은 무엇인가요?  ', 'Web service Monitoring System 2.0에서 제공하는 주요 기능은 어떤 것들이 있나요?  ', 'Web service Monitoring System 2.0의 기능 목록은 어떻게 되나요?  ', 'Web service Monitoring System 2.0의 주요 특징은 무엇인지 설명해 주세요.  ', 'Web service Monitoring System 2.0의 기능에 대해 자세히 알고 싶습니다.']


검색된 문서 개수: 5
또한 경보 설정을 통해 모니터링 등록된 URL에서 오류가 감지되면 SMS나 Email을 통해 알람 메시지를 수신할 수 있습니다.<그림3> WMS 2.0 Analytics Dashboard<그림4> WMS 2.0 Service DashboardWeb service Monitoring System 왜 필요한가요?웹 서비스의 사용자 경험을 극대화하기 위해서는 반드시 24/7 장애 및 이슈 없이 사용자가 웹 서비스를 이용할 수 있어야 합니다.우리는 대부분 웹 사이트의 사용자 경험에 대해 판단할 때 ‘깔끔한 경험 Flow, 정리된 콘텐츠, 이해하기 쉬운 서비스 이용 정책’ 등을 내세우고 있습니다. 하지만 무엇보다 중요한 것은 서비스가 24/7 이상 없이 작동하는 것입니다. 아무리 좋은 서비스를 제공한다고 하더라도 사용자가 원하는 시점에 정상적으로 작동하지 않는다면 사용자의 서비스에 대한 품질에 대한 인식이 저하될 수밖에 없기 때문입니다.또한, 서비스 동작 유무와 품질에 대한 모니터링이

Cloud Web Service Monitoring System(WMS) v2.0 A to Z | by Chang Hwan Jang | NAVER Cloud | MediumSitemapOpen in appSign upSign inMedium LogoWriteSign upSign inNAVER Cloud·네이버클라우드 기술 블로그 입니다.Cloud Web Service Monitoring System(WMS) v2.0 A to ZChang Hwan JangFollow20 min read·May 31, 2021--Share더욱 강력해진 웹 서비스 모니터링 기술의 이해<그림 1> WMS 2.0 Renewal UpdateSynthetic Monitoring vs Real User Monitoring웹 서비스의 모니터링 분야는 Synthetic Monitoring(가상 모니터링)과 Real User Monitoring(실제 사용자 모니터링)으로 나누어 분류할 수 있습니다.